In [ ]:
from glob import glob
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from src.vae_model import vae_model
from src.mapper_model import mapper_model
from src.util import plot_mesh_permarray
from src.util import seq_data

from pyeit import mesh


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
n_el = 16  # nb of electrodes
mesh_obj = mesh.create(n_el, h0=0.05)

In [ ]:
def load_all_data():
    voltage_dict = {} 
    gamma_dict = {} 
    data_dirs = sorted(glob("data*/"))  

    for i, directory in enumerate(data_dirs):
        file_list = sorted(glob(f"{directory}*.npz"))  
        voltage_list = []
        gamma_list = []

        for file in file_list:
            tmp = np.load(file, allow_pickle=True)  
            voltage_list.append(tmp["v"])  
            gamma_list.append(tmp["gamma"])  

        voltage_array = np.array(voltage_list) / 150
        voltage_array = np.expand_dims(voltage_array, axis=2)
        gamma_array = np.array(gamma_list) / 150          #WARUM Normalisierung?
        gamma_array = np.expand_dims(gamma_array, axis=2)
        
        # Speichere das Array im Dictionary
        voltage_dict[f"voltage{i}" if i > 0 else "voltage"] = voltage_array
        gamma_dict[f"gamma{i}" if i > 0 else "gamma"] = gamma_array
    
    return voltage_dict, gamma_dict

voltage_dict, gamma_dict = load_all_data()

voltage4 = voltage_dict["voltage4"]

gamma4 = gamma_dict["gamma4"]

In [ ]:
EIT = voltage4.reshape(-1, 32, 32, 1)
perm_array = perm_array.reshape(-1, 2840, 1)

In [ ]:
EIT_seq, perm_array_seq = seq_data(EIT, perm_array, n_seg=4)
# EIT_seq = EIT_seq.reshape(EIT_seq.shape[0], 4, 1024, 1)
print("New Shape of Voltages Data:")
print(EIT_seq.shape)
print("New Shape of Permittivity Data:")
print(perm_array_seq.shape)

In [ ]:
mapper = mapper_model()
mapper.summary()

mapper.compile(Adam(), loss="mse")